In [ ]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF

from src.SequentialSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection, SimpleCellposeSegmentaion

from src.FinalizationSteps import BuildPDFReport, SaveSpotDetectionResults, SaveMasksToAnalysis, SendAnalysisToNAS

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.GeneralOutput import OutputClass

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

# This code isn't compete, I need your help

In [ ]:
StepGUI().create_gui(BIGFISH_SpotDetection())


# Parameters
initalization

In [ ]:
scope = ScopeClass()
data = DataContainer()
settings = Settings(num_chunks_to_run=2, load_in_mask=True)
experiment = Experiment(initial_data_location=r'smFISH_images\Eric_smFISH_images\20230511\DUSP1_DexTimeConcSweep_10nM_75min_041223')




Individuals

In [ ]:
experiment.todict()

In [ ]:
experiment.nucChannel = 2
experiment.cytoChannel = 1

In [ ]:
experiment.todict()

All Params

In [ ]:
Parameters._instances

In [ ]:
Parameters.get_parameters()

Validate 

In [ ]:
Parameters.validate()

In [ ]:
experiment.FISHChannel = 0

In [ ]:
Parameters.validate()

# Load in data

In [ ]:
FFF2NativeDataType().run()

In [ ]:
data.todict()

chunks

In [ ]:
data.images

# Creating a step

In [ ]:
Display().displayImage_maxProject(0, 0, experiment.nucChannel)

In [ ]:
import cellpose
from cellpose import models, train
import numpy as np
import dask.array as da


model = models.CellposeModel(gpu=True, model_type='nuclei', pretrained_model=r"C:\Users\Jack\Documents\GitHub\FISH_Processing\models\GAPDH_cyto")

image = da.max(data.images[0, 0, experiment.nucChannel, :, :, :], axis=0)

image

In [ ]:
nuc_mask, flows, styles = model.eval(image, diameter=180, channels=[0, 0])

In [ ]:
plt.imshow(nuc_mask)

# Run Processing Steps

Cell Segmentation

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = [r"C:\Users\Jack\Documents\GitHub\FISH_Processing\models\GAPDH_cyto", r'C:\Users\Jack\Documents\GitHub\FISH_Processing\models\DAPI_nuclei']

In [ ]:
SimpleCellposeSegmentaion().run()

In [ ]:
OutputClass.get_all_instances()[0].__dict__

In [ ]:
Display().displayImage_maxProject(5, 0, channel=experiment.nucChannel)
Display().displayMask(5, 0, channel=experiment.nucChannel)

Spot Detection

In [ ]:
BIGFISH_SpotDetection().run()

Outputs

In [ ]:
spotDetection = OutputClass.get_all_instances()[1]
spotDetection

In [ ]:
vars(spotDetection)

In [ ]:
OutputClass.clear_instances()

# GUIs

In [ ]:
StepGUI().create_gui(SimpleCellposeSegmentaion())

In [ ]:
StepGUI().create_gui(BIGFISH_SpotDetection())


In [ ]:
OutputClass.clear_instances()

# Creating Pipelines

In [ ]:
FFF2NativeDataType()
SimpleCellposeSegmentaion()
BIGFISH_SpotDetection()

In [ ]:
pipeline = Pipeline()


In [ ]:
from src.GeneralStep import IndependentStepClass, StepClass, SequentialStepsClass

In [ ]:
print(IndependentStepClass._instances)
print(StepClass._instances)
print(SequentialStepsClass._instances)


In [ ]:
pipeline.execute_independent_steps()

In [ ]:
pipeline.execute_sequential_steps()

In [ ]:
pipeline.execute_sequential_steps()

# Running Multiple Pipelines Together

# Running on Clusters